<div class="alert alert-danger">
    <b>ACHTUNG</b>: Das Kapitel befindet sich noch in Arbeit und ist nicht fertig.
</div>

# Zusatzkapitel 2 - Neuronale Netze mit PyTorch und TorchText

## Z.2.1. Kapitelübersicht <a class="anchor" id="Z-2-1"/>

In diesem Kapitel wird gezeigt, wie man einfache Neuronale Netze mit der Deep-Learning-Bibliothek **PyTorch** und **TorchText** erstellt. Dieses Kapitel baut auf den Kapiteln 10 bis 13 auf, d.h. das Konzept von Neuronalen Netzen, gängige Deep Learning Begriffe und erste Praxiserfahrungen mit Keras sollten bekannt und vorhanden sein. Weiterhin sollte die Benutzung von Iteratoren in Python sowie das Konzept der objektorientierten Programmierung in Python geläufig sein, da PyTorch eine Klassenarchitektur zur Erstellung von Modellen benutzt. Auch wird die Benutzung einer GPU wie bei Kapitel 12 dringend empfohlen (auch hier kann wieder **Google Colab** benutzt werden).

TODO: Word Embeddings als Pflicht?

**Hinweis**: Das PyTorch-Modul befindet sich <u>nicht</u> in der bei dieser Tutorialreihe beigelegten `requirements.txt` Datei. Es muss eigenständig installiert werden, was jedoch im Gegensatz zu Keras oder Tensorflow weitaus unkomplizierter ist. Die Webseite von <a href="https://pytorch.org/">PyTorch</a> ist hier sehr hilfreich. <a href="https://github.com/pytorch/text">TorchText</a> muss ebenfalls noch installiert werden.

<b>Abschnittsübersicht</b>:<br>
[Z.2.1. Kapitelübersicht](#Z-2-1)<br>
[Z.2.2. Übersicht zu PyTorch](#Z-2-2)<br>
[Z.2.3. Vorverarbeitung des Korpus mit TorchText](#Z-2-3)<br>
[Z.2.3.1. Aufteilung des Korpus](#Z-2-3-1)<br>
[Z.2.3.2. Laden der Train-Val-Test-Dateien](#Z-2-3-2)<br>
[Z.2.3.3. Erstellung eines einfachen sequentiellen Modells TODO](#Z-2-4)<br>

Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- Implementierung eines Neuronalen Netz mit PyTorch

In [13]:
from nltk import word_tokenize
from nltk.corpus import stopwords
punctuation = ['!', '#','$','%','&', "'", '(',')','*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', 
               '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '`', '``', 'wurde', 'wurden']
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torchtext import data, datasets

## Z.2.2. Übersicht zu PyTorch <a class="anchor" id="Z-2-2"/>

<div class="alert alert-warning">
<b>Aufgabe:</b> Deep Learning with PyTorch <br>
    
Für das folgende Tutorial ist es hilfreich, begleitend das Buch von Eli Stevens et. al. "Deep Learning with PyTorch" zu lesen, welches einen guten Einstieg zu PyTorch liefert. 
    
<br><u>Das Buch</u>: STEVENS, Eli et. al., Deep Learning with PyTorch, July 2020 (englische Ausgabe).
<br><br>
    
Es eignet sich sehr gut für Leser, die bereits ersten Einblicke im Deep Learning gewonnen haben, wie etwa durch das Buch von Chollet (siehe Kapitel 10-12). Anders als bei dem Buch von Chollet wird das Buch von Stevens et. al. nicht für die Benutzung dieses Tutorials vorausgesetzt, es dient eher als Begleitlektüre mit vertiefenden, erweiterenden und ausführlicheren Behandlung von Themen. Die Autoren scheinen bei den Themen einen Fokus auf die Arbeit mit Bilddateien zu legen, weshalb ich im folgenden Kapitel hervorheben möchte, die für die Arbeit mit Textdaten hilfreich sind oder diese als Thema haben. Sollte man das Buch also nur überfliegen, sollten diese Kapitel mindestens oder ausführlicher gelesen werden:
- Kapitel 3: "It starts with a tensor". Gute Einführung zum Datentyp <b>Tensor</b>, der für PyTorch eine wichtige Rolle spielt.
- Kapitel 4.5: "Representing text". Zeigt, wie Textdaten mit PyTorch verarbeitet werden können und greift auch das Thema <b>Embeddings</b> auf.
- Kapitel 5: "The mechanics of learning". Hier werden noch einmal die Grundlagen von Neuronalen Netzen erklärt, aber auch einige PyTorch Eigenheiten, die wichtig sind.
- TODO
- ...

</div>

Wie auch **Keras** ist **PyTorch** eine Deep-Learning-Bibliothek, anders als Keras basiert PyTorch jedoch nicht auf der Bibliothek **Tensorflow**, sondern auf der Bibliothek **Torch**. PyTorch erfordert zudem eine stärkere Auseinandersetzung mit den Mechaniken von Deep Learning, weshalb auch mehr Code als bei Keras benötigt wird, in welchem meist mit nur wenigen Zeilen Code ein Neuronales Netz erstellt werden kann. Im akademischen Bereich wird PyTorch aktuell (Stand August 2020) jedoch viel häufiger als Keras verwendet, da es eine bessere Performance liefert und die Erstellung von komplexeren Neuronalen Netzwerkarchitekturen ermöglicht. Keras selbst wird anders als Tensorflow oder PyTorch aufgrund seiner Simplizität meistens nur zur Erstellung von Prototypen genutzt, abseits von Tutorials ist es deshalb seltener zu finden.

### PyTorch vs. Keras


<table align="left"> 
  <tr>
    <th></th>
    <th style="text-align:center;">Keras</th>
    <th style="text-align:center;">PyTorch</th>
  </tr>
  <tr>
      <td><b>Vorteile</b></td>
    <td>
        <ul style="text-align:left;">
          <li>Einsteigerfreundlich</li>
          <li>Simple Architektur</li>
          <li>Einfache Erstellung von Prototypen</li>
          <li>Gut für kleine Datensätze</li>
          <li>wenige Zeilen Code</li>
          <li>Debugging kaum notwendig</li>
        </ul> 
    </td>
    <td>
        <ul style="text-align:left;">
          <li>Sehr flexibel</li>
            <li>Höhere Perfomance</li>
            <li>Ermöglicht komplexere Neuronale Netzwerke</li>
            <li>Nutzt Python-Syntax</li>
            <li>Eignet sich gut fürs Debugging</li>
            <li>Gute Peformance auch für größere Datensätze</li>
            <li>Großer Community-Support</li>
            <li>Forschung nutzt vorrangig PyTorch, d.h. Paper nutzen PyTorch-Architektur</li>
        </ul>   
    </td>
  </tr>
  <tr>
    <td><b>Nachteile</b></td>
    <td>
        <ul style="text-align:left;">
          <li>Langsamer als PyTorch</li>
          <li>Benutzung der eigenen GPU kann umständlich werden</li>
            <li><i>Subjektiv:</i> Abgewandelte Python-Syntax</li>
            <li>Weniger Community-Support</li>
        </ul> 
    </td>
    <td>
        <ul style="text-align:left;">
          <li>Weniger einsteigerfreundlich als Keras</li>
          <li>Braucht viel mehr Code</li>
        </ul>    
    </td>
  </tr>
</table> 

Zu Beginn ist es recht nützlich, sich einen kurzen Überblick über die wichtigsten Module von PyTorch zu verschaffen, die im folgenden aufgelistet sind:

- `torch.nn`: Die Kern-Module, die zur Erstellung eines Neuronalen Netzes benötigt werden, befinden sich in diesem übergeordneten Modul. Vorwiegend befinden sich hier Module zur Erstellung von unterschieden **Layer** eines NN (z.B. fully connected Layer oder Convolutional Layer), aber auch **Aktivierungsfunktionen** und **Verlustfunktionen** können hiermit aufgerufen werden.
- `torch.optim`: Hier befinden sich Module zur Implementierung von **Optimierungsverfahren**. In Kapitel 12.5 wurden die Optimierungsverfahren *Adam*, *RMSprop* und *SGD* zur Hyperparameteroptimierung verwendet, diese stehen bei PyTorch ebenfalls zur Verfügung und können z.B. mit `from torch.optim as RMSprop` aufgerufen werden.
- `torch.tensor`: Dieses Modul befasst sich mit der Datenstruktur **Tensor**.
- `torch.utils.data`: Dieses Modul enthält die Klasse **`DataLoader`**, mithilfe derer Datensätze für die Verwendung von PyTorch geladen werden können.





TODO:<br>
- grund dinge wie torch modul & ähnliches erklären
- tabelle unten überpfüen und ggbf ergänzen



Sollte man mit Textdaten arbeiten, empfiehlt es sich, neben PyTorch noch die ebenfalls von PyTorch bereitgestelle Bibliothek **`TorchText`** zu verwenden. Diese vereinfacht typische Vorverarbeitungsschritte für die Arbeit mit Textdaten ungemein. Das Pendant für Bilddaten heißt **`TorchVision`** und das Pendant für Audiodaten heißt **`TorchAudio`**, beide Bibliotheken werden in diesem Tutorial jedoch nicht behandelt.

## Z.2.3. Vorverarbeitung des Korpus mit TorchText <a class="anchor" id="Z-2-3"/>

Das Einlesen und die Aufteilung des Korpus unterscheidet sich von dem Vorgehen in den anderen Kapiteln. PyTorch erfordert mehr noch als Keras eine ausführlichere Vorverarbeitung des Korpus. Für das Einlesen und für die Vorverarbeitung von Textdaten ist es daher sinnvoll, die Bibliothek **`TorchText`** zu verwenden. `TorchText` hilft bei den folgenden Operationen[<sup>1</sup>](#fn1):

- **Laden von Dateien**: Laden in das Korpus aus verschiedenen Formaten
- **Tokenisierung**: Sätze in Wortlisten aufteilen
- **Vokabular erstellen**: Erstellen einer Vokabelliste
- **Numerisieren/Indexifizieren**: Wörter eines Korpus als Ganzzahlen darstellen
- **Wortvektoren**: entweder das Vokabular zufällig initialisieren oder aus einem vortrainierten Embedding laden (dieses muss jedoch "getrimmt" werden, d.h. speichern nur Wörter aus den Vokabular werden auch gespeichert).
- **Batching**: Erzeugen von Batches von Trainingsdaten


Leider übernimmt TorchText nicht die folgenden Aufgaben, die manuell oder mit anderen Hilfsbibliotheken durchgeführt werden müssen:

- **Train-Val-Test-Split**: Korpus in Trainings-, Validierungs und Testdaten aufteilen
- **Embedding Lookup**: Zuordnung jedes Satzes (der Wortindizes enthält) zu Wortvektoren einer festen Dimension, welches für Word Embeddings relevant ist


<hr style="border: 0.1px solid black;"/>
<div id="fn1" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">1</sup>&nbsp; Die Liste wurde von diesem <a href="https://anie.me/On-Torchtext/"> Blogpost</a> entnommen.</div>

### Z.2.3.1. Aufteilung des Korpus <a class="anchor" id="Z-2-3-1"/>

Viele Klassen und Methoden von PyTorch und TorchText erwarten verschiedene Dateien für den Trainings-, Test- und Validierungsdatensatz. Mithilfe der uns bereits bekannten `train_test_split`-Funktion von Scikit-learn können wir diese Dateien erstellen, da die Funktion als Eingabe neben numpy-Matrizen auch DataFrames erlaubt. Bis jetzt hatten wir den Datei-Typ **csv** verwendet. Dieser ist jedoch für die Nutzung mit TorchText problematisch, die JSON Variante <a href="http://jsonlines.org/">JSON Lines</a> eignet sich hier besser.[<sup>2</sup>](#fn2) Beim **JSON Line** Format ist jede Zeile ein valider JSON Wert. Die gesplitteten Datensätze des Wikipedia-Korpus wandeln wir mit einer selbstgeschriebenen Funktion `df_to_jsonl` in das JSON Line Format um. Zuvor wurden für dieses Tutorial die Spalten "id" und "length" aus dem Korpus entfernt.


<hr style="border: 0.1px solid black;"/>
<div id="fn2" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">2</sup>&nbsp; Unter anderem liegt das daran, dass csv-Dateien mit Kommas (oder Semikolons) voneinander getrennt werden und TorchText Probleme hat, diese Trennungskommas von Kommas in der "Text"-Spalte zu unterschieden. Für weitere Erklärungen zum Vorteil von JSON-Dateien gegenüber csv-Dateien siehe dieses <a href="https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/A%20-%20Using%20TorchText%20with%20Your%20Own%20Datasets.ipynb">TorchText-Tutorial</a>.</div>

In [14]:
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col = 0)
corpus = corpus.drop("id", axis=1)
corpus = corpus.drop("length", axis=1)

In [15]:
corpus.head()

,category,text
0,Album nach Typ,All the Best ! ( englisch Alles Gute ! ) ist d...
1,Album nach Typ,Let It Roll : Songs by George Harrison ist das...
2,Album nach Typ,Lieder wie Orkane ist das dritte offizielle Be...
3,Album nach Typ,Long Stick Goes Boom : The Anthology ist eine ...
4,Album nach Typ,Los Grandes Éxitos en Español ( spanisch für D...


In [16]:
X_train, X_remain = train_test_split(corpus, 
                                     test_size=0.4, 
                                     train_size=0.6,
                                     random_state=42,
                                     stratify=corpus["category"])
X_val = X_remain[:1200]
X_test = X_remain[1200:]


def df_to_jsonl(df, filename, column="text", path="tutorialdata/pytorch_data/"):
    """ DataFrame with text column to Json Line Format. """

    df[column] = df.apply(lambda row: word_tokenize(row[column]), axis=1)
    df.to_json(f"{path}{filename}.json", orient='records', lines=True)
    
    
df_to_jsonl(X_train, "train")
df_to_jsonl(X_val, "val")
df_to_jsonl(X_test, "test")

/Users/jan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


### Z.2.3.2. Laden der Train-Val-Test-Dateien <a class="anchor" id="Z-2-3-2"/>

Eines der Hauptkonzepte von TorchText ist das sogenannte **`Field`**. Dadurch wird definiert, wie Daten verarbeitet werden sollen. So kann z.B. angegeben werden, ob Texte in diesem `Field` tokenisiert und nur in Kleinbuchstaben dargestellt werden sollen. 

Im `Field` `TEXT` in der folgenden Code-Zelle wurde als Tokenizer *toktok* angegeben, eine Tokenizer der NLP-Bibliothek **NLTK** (andere mögliche Werte finden sich <a href="https://pytorch.org/text/_modules/torchtext/data/utils.html">hier</a>). In unserem Fall ändert sich durch die Angabe des Tokenizers nichts an den Texten der Datensatz-Dateien, da diese bereits vorher wegen der Konvertierung ins JSON Lines Format tokenisiert wurden.[<sup>3</sup>](#fn3) Weiterhin haben wir alle Wörter mit dem Argument `lower` in Kleinbuchstaben konvertiert.[<sup>4</sup>](#fn4) Auch Stoppwörter sowie viele Satzzeichen wurden ignoriert.<br>




<hr style="border: 0.1px solid black;"/>
<div id="fn3" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">3</sup>&nbsp; Texte sollten im besten Fall vorher tokenisiert werden, da ansonsten TorchText jedes Mal, wenn die Datensätze geladen werden, diese neu tokenisiert. Dies kann bei großen Datensätzen sehr viel Zeit kosten.</div>
<div id="fn4" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">4</sup>&nbsp; Dies ist für die deutsche Sprache nicht immer sinnvoll, wurde hier jedoch verwendet, um ein reichhaltigeres Vokabular mit weniger doppelten Wörtern zu erhalten.</div>

In [17]:
stop_words = stopwords.words('german') + punctuation 

TEXT = data.Field(tokenize = "toktok",
                  lower = True,
                  stop_words=stop_words)

Neben der Basisklasse gibt es von `Field` noch verschiedene Varianten. Für die Spalte mit den Klassen verwenden wir beispielsweise die Variante **`LabelField`**. Im Grunde unterscheidet diese sich nicht groß von `Field`, jedoch sind die Argumente `sequential` und `unk_token` automatisch auf `None` gesetzt. Dies macht Sinn, da es sich bei unseren Labeln anders als beim `Field` `TEXT` nicht um sequentielle Daten (*hier*: keine ganzen Texte) handelt und es auch nicht zu einem Out-of-Vocabulary (OOV) Fehler kommen kann.

In [18]:
CATEGORIES = data.LabelField()

Mit der Funktion `data.TabularDataset.splits` rufen wir die verschiedenen JSON-Dateien als Tabellen-Datensatz auf. Dem Argument `fields` übergeben wir ein Dictionary `assigned_fields`, in welchen wir unseren erstellten `Fields` `TEXT` und `CATEGORIES` die entsprechenden zugehörigen Spalten unserer Datensätze zuordnen. In der späteren Trainingsschleife können wir uns das zu Nutze machen und z.B. über `batch.category` auf die Kategorien zugreifen. Mit dem `skip_header = True` geben wir an, dass die Spaltenbezeichnungen ignoriert werden.

In [19]:
assigned_fields = {"text": ('text', TEXT), 
                   "category": ('category', CATEGORIES)}

train_data, val_data, test_data = data.TabularDataset.splits(path='tutorialdata/pytorch_data/', 
                                                              train='train.json',
                                                              validation='val.json', 
                                                              test='test.json', 
                                                              format='json',
                                                              fields=assigned_fields,
                                                              skip_header = True)

In [20]:
example = vars(train_data.examples[1])
print(example["text"][:20])

['satz', 'trachtenbrot', 'benannt', 'boris', 'trachtenbrot', 'satz', 'mathematischen', 'logik', '1950', 'bewiesen', 'besagt', 'endlichen', 'modellen', 'allgemeingültigen', 'sätze', 'prädikatenlogik', 'erster', 'stufe', 'rekursiv', 'aufzählbar']


#### Begrenzung des Vokabulars

Nun geben wir die maximale Größe unseres Vokabulars an, d.h. wir begrenzen die von unserem Modell zu verarbeitenden Wörter auf die 25000 häufigsten Wörter. Dieser Wert kann je nach Belieben/vorhandener Rechenkraft/Fragestellung erhöht oder verringert werden. Geben wir jedoch die tatsächliche Größe unseres Vokabulars aus, nachdem wir es mit `build_vocab` erstellt haben, erhalten wir den Wert 25002. Dies liegt daran, dass noch die zwei zusätzlichen Vokabeln `<unk>` und `<pad>` hinzugefügt werden: Da wir nur das Vokabular unseres Trainingsdatensatzes kennen, kann es sein, dass es im Validierungs- oder Testdatensatz Wörter gibt, die nicht in unserem Vokabular vorkommen. Um nicht den OOV-Fehler zu erhalten, wird von PyTorch die Vokabel `<unk>` verwendet (= unknown). Der `<pad>`-Token (= padding) wird zum "Auffüllen" von Batches verwendet.[<sup>5</sup>](#fn5) <br>


<hr style="border: 0.1px solid black;"/>
<div id="fn5" style="font-size:8pt; line-height:1; padding-left: 1em; text-indent: -1em"><sup style="font-size:5pt">5</sup>&nbsp; Die GPU (oder CPU) verarbeitet die Trainingsdaten beim Training des Neuronalen Netzes in <b>batches</b> (= Stapel), die alle eine gewisse Länge haben. Ist ein Batch zu klein, wird er mit Padding-Tokens aufgefüllt, damit alle Batches die gleiche Länge haben.</div>

In [21]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
CATEGORIES.build_vocab(train_data)

In [22]:
print(f"Einzigarte Tokens im TEXT-Vokabular: {len(TEXT.vocab)}")
print(f"Einzigarte Tokens im CATEGORIES-Vokabular: {len(CATEGORIES.vocab)}")

Einzigarte Tokens im TEXT-Vokabular: 25002
Einzigarte Tokens im CATEGORIES-Vokabular: 30


In [23]:
print(TEXT.vocab.freqs.most_common(20))

[('jedoch', 2720), ('sowie', 2706), ('jahr', 2678), ('the', 2566), ('2', 2386), ('seit', 2320), ('zwei', 2272), ('1', 2258), ('stadt', 2243), ('ab', 2203), ('jahre', 2103), ('mehr', 2056), ('etwa', 1994), ('zeit', 1963), ('of', 1826), ('ersten', 1820), ('dabei', 1712), ('jahren', 1677), ('drei', 1671), ('a', 1660)]


Das Vokabular kann auch direkt angezeigt werden, indem wir die Funktionen `stoi` (= string to int) oder `itos` (= int to string) verwenden.

In [24]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'jedoch', 'sowie', 'jahr', 'the', '2', 'seit', 'zwei', '1']


Der letzte Schritt bei der Vorbereitung der Daten ist die Erstellung der **Iteratoren**. Ein Datensatz-Iterator ermöglicht das einfache Laden von Daten in Neuronalen Netzwerke und hilft bei der Organisation von Stapelverarbeitung (= batching), Maskierung und Konvertierung (z.B. die Wörter durch die Indexnummer der Wörter zu ersetzen). Wir iterieren über diese Iteratoren in der Trainings-/Evaluierungsschleife und sie geben bei jeder Iteration einen Batch von Daten zurück (indiziert und in Tensoren umgewandelt). Dafür verwenden wir den sogenannten `BucketIterator`, der einen Batch von Daten zurückgibt, bei der jeder Datenpunkt eine ähnliche Länge hat. Somit wird der Gebrauch von Padding (*hier*: den `<pad>` Tokens) minimiert. Sollte eine GPU vorhanden und CUDA erfolgreich installiert worden sein, kann mithilfe von `torch.device` diese genutzt werden, indem sie `device` dem `BucketIterator` übergeben wird. Zuletzt geben wir ähnlich wie bei Keras noch die Größe der Batches an.

TODO: argumente

In [25]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, val_iterator, test_iterator = data.BucketIterator.splits((train_data, val_data, test_data), 
                                                                         batch_size = BATCH_SIZE,
                                                                         device = device,
                                                                         sort_key = lambda x: len(x.text),
                                                                         sort = False,
                                                                         sort_within_batch=False)

Auf diese Iteratoren können wir nun zugreifen. Wir erstellen hier zu Demonstrationszwecken einen Batch des `train_iterator` außerhalb der eigentlichen Schleife. Mithilfe von `batch.text` können wir uns nun unsere umgewandelten Textdaten anzeigen lassen, die indiziert wurden und in einem Tensor gespeichert wurden. Dabei wird auf das erstellte Dictionary `assigned_fields` zugegriffen, genauer auf den Namen im Tuple zum Key "text". Wir konnten dort einen beliebigen Namen wählen, auf den wir jetzt zugreifen. Mit `batch.category` können wir auch auf unsere Kategorien zugreifen, die ebenfalls indiziert und in Tensoren umgewandelt wurden. Das dies erfolgreich war, können wir daran erkennen, dass nur Zahlen zwischen 0 und 29 in unserem Tensor vorkommen, also 30 verschiedene Indizes. Da wir 30 verschiedene Kategorien haben, passt dies überein.

In [29]:
batch = next(iter(train_iterator))
print(batch.text)

tensor([[  994, 10200,     5,  ...,     5,   556,     0],
        [    0, 12038, 15465,  ...,  2741,     0,     0],
        [ 2510,  9428,  2314,  ...,    16,     0,     0],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])


In [30]:
print(batch.category)

tensor([ 1,  4,  7, 10, 20, 21, 25,  8, 29, 23,  4,  9, 21, 26, 22, 16, 27, 18,
         3, 12,  2,  2,  8,  6, 11, 10, 14, 14, 28, 13,  8,  1, 22, 17, 13, 26,
        18, 20,  8,  5, 18, 24, 17, 21, 27,  2, 20,  6, 18, 21, 24,  0, 23, 23,
        13, 15,  0,  0, 12, 14, 22,  0, 21, 17])


## Z.2.4. Erstellung eines einfachen sequentiellen Modells TODO<a class="anchor" id="Z-2-4"/>

Anders als bei Keras stützt sich das Erstellen von Modellen auf Klassen, die wir selbst erstellen müssen. Dadurch wird die Menge an Code im Gegensatz zu Keras erhöht, dafür können wir auch viel besser unser Neuronales Netz auf unsere Bedürfnisse anpassen. Wir erhalten hier <a href="https://en.wikipedia.org/wiki/Boilerplate_code">Boilerplate Code</a>. Für die Erstellung eines einfachen sequentiellen Modells in Keras brauchten wir nur folgenden Code:<br>

```
model = models.Sequential()
model.add(layers.Dense(64, activation="relu", input_shape=(len(vocab),)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(len(np.unique(labels)), activation="softmax"))

model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])
```

Für ... TODO

TODO: vllt erstmal folgende bibliothek benutzen: https://github.com/ncullen93/torchsample

damit wird training einfach wie bei keras!!!

In [189]:
input_shape = len(TEXT.vocab)
output_shape = len(CATEGORIES.vocab)

In [31]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):

        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):

        #text = [sent len, batch size]
        embedded = self.embedding(text)
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))

        return self.fc(hidden.squeeze(0))

In [43]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = len(CATEGORIES.vocab)

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [44]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [47]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        predictions = model(batch.text)
        loss = criterion(predictions, batch.category)
        acc = categorical_accuracy(predictions, batch.category)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)
            loss = criterion(predictions, batch.category)
            acc = categorical_accuracy(predictions, batch.category)
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 9m 17s
	Train Loss: 3.428 | Train Acc: 3.46%
	 Val. Loss: 3.415 |  Val. Acc: 2.39%


#### 1. Ansatz: `nn.Sequential()`

Dieser erste Ansatz ähnelt sehr dem Aufbau des Keras-Modells. In der Praxis wird dieser Ansatz seltener benutzt, weshalb wir uns eher auf den nächsten Ansatz fokussieren werden. Für eine ersten Einstieg in die Modell-Architektur von PyTorch ist es aber sehr hilfreich. Ähnlich wie bei Keras definieren wir das sequentielle Modell mit `nn.Sequential`. Diesem übergeben wir eine Liste von Layern, in Keras wurden diese nach und nach hinzugefügt. Wir können alternativ dem sequentiellen Modell auch direkt unsere Layer übergeben, etwa so:<br>

```
model = nn.Sequential(nn.Linear(input_shape, 64),
                      nn.ReLU(),
                      ...)
```

Die Übergabe einer Liste von Layern ist m.E. jedoch übersichtlicher. Bei den Layern gibt es jedoch einige Unterschiede zu Keras. Das Äquivalent zum Dense-Layer in Keras ist in PyTorch der Linear-Layer. Die Aktivierungslayer werden anders als im oben gezeigten Keras-Code nicht innerhalb des Dense-Layer angegeben, sondern als eigenständiger Layer. Jeder Linear-Layer hat zwei Parameter: Die Dimension der Eingabedaten und die Dimension der Ausagebdaten. Im ersten Layer ist die Eingabedimension gleich der Größe unseres Text-Vokabulars, im letzten Layer ist die Ausgabedimension gleich der Anzahl unserer Kategorien. Die anderen Parameter beziehen sich auf die Dimensionen des vorherigen bzw. nächsten Layers. So muss die Eingabedimension eines Layers immer gleich der Ausgabedimension des vorherigen Layers sein. Dies wurde von Keras automatisch verwaltet, in PyTorch müssen wir dies selbst angeben.

In [190]:
layers = []
layers.append(nn.Linear(input_shape, 64))
layers.append(nn.ReLU())
layers.append(nn.Linear(64, 64))
layers.append(nn.ReLU())
layers.append(nn.Linear(64, output_shape))
layers.append(nn.Softmax())

model = nn.Sequential(*layers)
print(model)

Sequential(
  (0): Linear(in_features=25002, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=30, bias=True)
  (5): Softmax(dim=None)
)


#### 2. Ansatz: `nn.Sequential()` + `torch.nn.Model` Klasse

TODO

##### Sequential

In [201]:
class SequentialModel(nn.Module):
    def __init__(self, 
                 input_shape,
                 embedding_dim,
                 hidden_dim,
                 output_shape):
        #super(SequentialModel, self).__init__()
        super().__init__()
        """
        self.layers = nn.Sequential(nn.Linear(input_shape, hidden_dim1),
                                    nn.ReLU(),
                                    nn.BatchNorm1d(hidden_dim1),
                                    nn.Linear(hidden_dim1, hidden_dim2),
                                    nn.ReLU(),
                                    nn.BatchNorm1d(hidden_dim2),
                                    nn.Linear(hidden_dim2, output_shape),
                                    nn.Softmax(dim=output_shape)
                                   )
        """
        self.embedding = nn.Embedding(input_shape, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_shape)

    def forward(self, x):
        
        # TODO: fixen
        
        #x = nn.Softmax(self.layers)
        #return x
        #x = x.view(x.size(0), -1)
        # return self.layers(x)
        """
        print(f"linear1 weights: {self.linear1.weight.size()}")
        x = self.linear1(x)
        hidden_relu = nn.functional.relu(x)
        y_pred = self.out(x)
        return y_pred
        """
        
        #print(f"x shape: {x.shape}")
        embedded = self.embedding(x)
        linear = self.linear1(embedded)
        #output, hidden = self.linear1(embedded)
        
        #assert torch.equal(linear[-1,:,:], linear.squeeze(0))
        
        return self.out(linear.squeeze(0)) 

##### RNN

In [203]:
class RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):
        
        #print(text.shape)
        
        #text = [sent len, batch size]
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [204]:
#model = SequentialModel(input_shape, 100, 64, output_shape)
model = RNN(input_shape, 100, 256, output_shape)

In [205]:
print(model)

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=30, bias=True)
)


In [206]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# RuntimeError: Expected object of device type cuda 
# but got device type cpu for argument #1 'self' in call to _th_addmm
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)
        
        loss = criterion(predictions, batch.category)
        
        acc = categorical_accuracy(predictions, batch.category)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [207]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.category)
            
            acc = categorical_accuracy(predictions, batch.category)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [208]:
N_EPOCHS = 4

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)
    
    
    print(f'Epoch: {epoch+1} ')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 1 
	Train Loss: 3.441 | Train Acc: 3.00%
	 Val. Loss: 3.419 |  Val. Acc: 3.43%
Epoch: 2 
	Train Loss: 3.426 | Train Acc: 3.37%
	 Val. Loss: 3.412 |  Val. Acc: 3.60%


KeyboardInterrupt: 

In [129]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [128]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


# RuntimeError: Expected object of device type cuda 
# but got device type cpu for argument #1 'self' in call to _th_addmm
model = model.to(device)
criterion = criterion.to(device)

for epoch in range(2):
    
    model.train()
    print(f"Epoch: {epoch}")
    
    
    for batch in train_iterator:
        optimizer.zero_grad()
        
        y_pred = model(batch.text)
        

Epoch: 0
Epoch: 1


In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,606,302 trainable parameters


TODO:
- https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/5%20-%20Multi-class%20Sentiment%20Analysis.ipynb
- acc berechnen
- train funktion erstellen
- evaluate funktion erstellen

In [21]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().cuda()

model = model.to(device)
criterion = criterion.to(device)

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [22]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text.float())
        
        loss = criterion(predictions, batch.category)
        
        acc = categorical_accuracy(predictions, batch.category)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [23]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)
            
            loss = criterion(predictions, batch.category)
            
            acc = categorical_accuracy(predictions, batch.category)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [25]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    """
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut5-model.pt')
    """
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')



RuntimeError: size mismatch, m1: [1270 x 64], m2: [25002 x 64] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:290

In [ ]:
input_shape = len(TEXT.vocab)
output_shape = len(CATEGORIES.vocab)
model = SequentialModel(input_shape, output_shape)

In [ ]:
print(model)

In [ ]:
class Model(nn.Module):
    def __init__(self, )

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

TODO: https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

TODO word embeddings: https://www.innoq.com/en/blog/handling-german-text-with-torchtext/